In [ ]:
import numpy as np 
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import joblib
import json
import dice_ml

In [ ]:
tfmodel = tf.keras.models.load_model('../models/adult_NN/')

In [ ]:
dataset = pd.read_csv('../data/adult_prep_test.csv')
target = dataset['income']

train_dataset, test_dataset, _, _ = train_test_split(dataset,
                                                     target,
                                                     test_size=0.2,
                                                     random_state=0,
                                                     stratify=target)
train_dataset

In [ ]:
columns = train_dataset.columns.tolist()
columns.remove('income')

# Dataset for training an ML model
d = dice_ml.Data(dataframe=train_dataset,
                 continuous_features=columns,
                 outcome_name='income')

# # Pre-trained ML model
m = dice_ml.Model(model=tfmodel, backend='TF2', func="ohe-min-max")

# DiCE explanation instance
exp = dice_ml.Dice(d, m)

# Generate counterfactual examples
query_instance = test_dataset.drop(columns="income")[10:11]

In [ ]:
columns_list = train_dataset[columns].columns.to_list()

# Non-actionable features
nonactionable_features = []
nonactionable_features.append('age') 
nonactionable_features += list(filter(lambda x: 'native' in x, columns_list))
nonactionable_features += list(filter(lambda x: 'sex' in x, columns_list))
nonactionable_features += list(filter(lambda x: 'race' in x, columns_list))

# Mask for non-actionable features
mask_indices = [columns.index(feat) for feat in nonactionable_features]

In [ ]:
features_to_vary = list(set(columns_list) - set(nonactionable_features))
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=60, desired_class="opposite", features_to_vary=features_to_vary)

counterfactuals = json.loads(dice_exp.to_json())['cfs_list'][0]
counterfactuals